# Thinking

## ALS都有哪些应用场景（能简单说明ALS的应用场景）
    1、ALS算法用来补全用户评分矩阵。由于用户评分矩阵比较稀疏，将用户评分矩阵进行分解，变成V和U的乘积。通过求得V和U两个小的矩阵来补
      全用户评分矩阵。
    2、ALS算法使用交替最小二乘法来进行求解。
    3、ALS分为显示反馈和隐式反馈两种。显示反馈是指用户有明确的评分。对于商品推荐来说，大部分是通过用户的行为，获取隐式反馈的评分。
       隐式反馈评分矩阵需要进行处理，如果有用户评分则置为1，没有则赋值为0。但是对这个处理后的评分矩阵，再有一个置信度来评价这个评分。
       置信度等于1+a*用户真实评分
    4、ALS的代价函数是估计值和现有的评分值误差的平方和，引入了L2正则


## ALS进行矩阵分解的时候，为什么可以并行化处理（理解ALS的原理，并能用自己的语言解释ALS能并行化的原因）
    这里的并行计算是值不同特征向量X、Y之间的并行计算,最小二乘法是通过最小化误差的平方和来寻找和数据最匹配的函数。使用交替最小二乘法
    来求解。步骤是先设置一个特征值X，然后求解另一个特征矩阵Y。然后再固定一个矩阵Y，求解另一个矩阵X。这就是交替二乘法的步骤。在矩阵
    求解的过程中，比如固定Y，求解X的话，目标评分矩阵A。X的每一行可以独立求解，X的第i行和Y的计算得到A的第i行。这样的话，对于每一步来说，
    X或者Y的行或者列都是可以独立并行求解的。这样ALS就可以进行并行化计算了。

## 梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD）（能说明三种梯度下降的区别）
    BGD:每次迭代时使用所有的样本来更新参数,速度慢，但迭代次数少；可以将所有的样本放在一个矩阵中，实现并行计算,在整个数据集上确定梯度
        下降的方向，当目标函数为凸函数时，BGD一定可以达到全局最优。
    SGD:每次迭代使用一个样本来更新参数，速度快，但是不利于实现并行操作，容易陷入局部最优，因为单个样本不能代表整体样本的趋势
    MBGD:它是BGD和SGD的综合，每次迭代使用一批样本来更新参数

## 你阅读过和推荐系统/计算广告/预测相关的论文么？有哪些论文是你比较推荐的，可以分享到微信群中 （论文推荐分享）

# Action

## 对MovieLens数据集进行评分预测工具：可以使用Surprise或者其他说明使用的模型，及简要原理（1、完成代码，流程正确2、能使用Surprise或其他工具，并对使用模型的原理进行说明）




In [112]:
#加载相关包
# !pip install surprise
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic,NormalPredictor
from surprise import KNNBaseline
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd 




# reg_i：物品的正则化参数，默认为10。
# reg_u：用户的正则化参数，默认为15 。
# n_epochs：迭代次数，默认为10
# 使用als方式

# 加载数据
def load_data():
    reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
    data = Dataset.load_from_file('./code/MovieLens/ratings.csv', reader=reader)
    return data
#把数据转换成dataframe
def data_to_df(data):
    raw_ratings = data.raw_ratings
    #用户id
    userId=[]
    #电影id
    itemId=[]
    #评分
    rate=[]
    for raw in raw_ratings:
        userId.append(raw[0])
        itemId.append(raw[1])
        rate.append(raw[2])
    data_dit={"userId":userId,"itemId":itemId,"rate":rate}
    return pd.DataFrame(data_dit)
    
    
# 通过globals()方法对传入对字符串使用baseline方式，KNNBAseLine方式，以及NormalPredictor方式对评分预测
# NormalPredictor 认为用户对物品的评分是服从正态分布的，从而可以根据已有的评分的均值和方差 预测当前用户对其他物品评分的分数 
#                 评分数据是来自一个正态分布的数据，根据训练集的分布特征随机给出一个预测值
# BaselineOnly 由于不同用户评分不同，不同item评分也不同，不同时间评分也可能发生变化，所以该算法通过设立基线(用户对item评分的平均值)，
#              并引入用户的偏差以及item的偏差
# KNNBasic 是基于 一种协同过滤方法
# KNNBaseline 对于用户和物品的评分有高低，每个用户打分均值或者每个item打分的均值，去除基于baseline偏差运算的分数
def predict_rate(data,df,methods,bsl_options = None):
    print("采用{}方式预测，参数为{}".format(methods,bsl_options))
    algo = None
    if(bsl_options):
        algo = globals()[methods](bsl_options)
    else:
        algo = globals()[methods]()
    data_set = data.build_full_trainset()
    kf = KFold(n_splits=3)
    for trainset, testset in kf.split(data):
        algo.fit(trainset)
        predictions = algo.test(testset)
        accuracy.rmse(predictions)
#     rank = globals()[methods](user, N)
    user_items=[("196","302"),("196","592"),("196","709"),("196","1088"),("196","2846")]
    for user_item in user_items:
        pred = algo.predict(user_item[0], user_item[1], r_ui=4)
        userItem=df[(df["userId"]==user_item[0])&(df["itemId"]==user_item[1])]
        userItem=list(userItem["rate"]) 
        if len(userItem)>0:
            print('用户{}对电影{}的评分预测为{},实际得分为{}'.format(user_item[0],user_item[1],pred.est,userItem[0])) 
        else :
            print('用户{}对电影{}的评分预测为{}'.format(uid,iid,pred.est)) 
    return
    

# algo = NormalPredictor()
#     bsl_options = {'method': method,'n_epochs': 5,'reg_u': 12,'reg_i': 5}

data =load_data()
df=data_to_df(data)
print('NormalPredictor第一次运行')
predict_rate(data,df,methods='NormalPredictor')
print('NormalPredictor第一次运行结束')
print('NormalPredictor第二次运行')
predict_rate(data,df,methods='NormalPredictor')
print('NormalPredictor第二次运行结束')

bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
print('BaselineOnly的als第一次运行')
predict_rate(data,df,methods='BaselineOnly',bsl_options=bsl_options)
print('BaselineOnly的als第一次运行结束')
print('BaselineOnly的als第二次运行')
predict_rate(data,df,methods='BaselineOnly',bsl_options=bsl_options)
print('BaselineOnly的als第二次运行结束')

bsl_options = {'method': 'sgd','n_epochs': 5}
print('BaselineOnly的method第一次运行')
predict_rate(data,df,methods='BaselineOnly',bsl_options=bsl_options)
print('BaselineOnly第一次运行结束')

print('BaselineOnly的sgd第二次运行')
predict_rate(data,df,methods='BaselineOnly',bsl_options=bsl_options)
print('BaselineOnly的sgd第二次运行结束')



print('KNNBasic第一次运行')
predict_rate(data,df,methods='KNNBasic')
print('KNNBasic第一次运行结束')

print('KNNBasic第二次运行')
predict_rate(data,df,methods='KNNBasic')
print('KNNBasic第二次运行结束')


print('KNNBaseline第一次运行')
predict_rate(data,df,methods='KNNBaseline')
print('KNNBaseline第一次运行结束')

print('KNNBaseline第二次运行')
predict_rate(data,df,methods='KNNBaseline')
print('KNNBaseline第二次运行结束')






NormalPredictor第一次运行
采用NormalPredictor方式预测，参数为None
RMSE: 1.4316
RMSE: 1.4341
RMSE: 1.4328
用户196对电影302的评分预测为1.9324761555437031
用户196对电影592的评分预测为4.085282160843213,实际得分为4.0
用户196对电影709的评分预测为4.637768700542509,实际得分为3.0
用户196对电影1088的评分预测为3.379558528171796,实际得分为5.0
用户196对电影2846的评分预测为3.877383675920515,实际得分为2.0
NormalPredictor第一次运行结束
NormalPredictor第二次运行
采用NormalPredictor方式预测，参数为None
RMSE: 1.4320
RMSE: 1.4330
RMSE: 1.4336
用户196对电影302的评分预测为4.122559863095595
用户196对电影592的评分预测为2.9554838094242544,实际得分为4.0
用户196对电影709的评分预测为5,实际得分为3.0
用户196对电影1088的评分预测为4.138764257794863,实际得分为5.0
用户196对电影2846的评分预测为3.116690515426584,实际得分为2.0
NormalPredictor第二次运行结束
BaselineOnly的als第一次运行
采用BaselineOnly方式预测，参数为{'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5}
Estimating biases using als...
RMSE: 0.8617
Estimating biases using als...
RMSE: 0.8650
Estimating biases using als...
RMSE: 0.8645
用户196对电影302的评分预测为4.168211921957271
用户196对电影592的评分预测为3.8189558842329996,实际得分为4.0
用户196对电影709的评分预测为3.705614290389705,实际得分为3.0
用户19

从上面的运行结果可以看出，NormalPredictor的预测评分随机性比较大，与用户实际评分差距也比较大；BaselineOnly的稳定性比较好
KNNBasic与KNNBaseline的运行效率很差，但是他们可以与用户评分最接近，用他们更可以反应用户对该item的喜好度

## Paper Reading：Slope one predictors for online rating-based collaborative filtering. Daniel Lemire and Anna Maclachlan, 2007. http://arxiv.org/abs/cs/0702144.积累，总结笔记，自己的思考及idea （1、完成论文阅读，整理阅读笔记 2、有自己的idea）


## 设计你自己的句子生成器 （完成代码，结果正确）
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''


In [123]:
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''
import random

# 得到语法字典
def getGrammarDict(gram, linesplit = "\n", gramsplit = "=>"):
    #定义字典
    result = {}

    for line in gram.split(linesplit):
        # 去掉首尾空格后，如果为空则退出
        if not line.strip(): 
            continue
        expr, statement = line.split(gramsplit)
        # =>前面的字符串作为key   对statement通过｜分割，把分割后内容通过空格分割，这样形成字典
        result[expr.strip()] = [i.split() for i in statement.split("|")]
#     print(result)
    return result


# 生成句子
# 句子内容格式已经确定 套用公式 如果为战斗 将会形成两句话 主要包含了施法 和 结果部分
# 施法部分是用 语 动作 技能组成的
# 结果部分是由 主语 获得 效果组成
# 如此依次寻找每个词语在字典中是否存在key，如果不存在则为语句组成部分，如果存在则需要通过递归方式继续往下找数组中随机出来的词语
def generate(gramdict, target, isEng = False):
    if target not in gramdict: 
        return target
    find = random.choice(gramdict[target])
    blank = ''
    # 如果是英文中间间隔为空格
    if isEng: 
        blank = ' '
    return blank.join(generate(gramdict, t, isEng) for t in find)

gramdict = getGrammarDict(grammar)
print(generate(gramdict,"战斗"))
print(generate(gramdict,"战斗", True))






{'战斗': [['施法', '，', '结果', '。']], '施法': [['主语', '动作', '技能']], '结果': [['主语', '获得', '效果']], '主语': [['张飞'], ['关羽'], ['赵云'], ['典韦'], ['许褚'], ['刘备'], ['黄忠'], ['曹操'], ['鲁班七号'], ['貂蝉']], '动作': [['施放'], ['使用'], ['召唤']], '技能': [['一骑当千'], ['单刀赴会'], ['青龙偃月'], ['刀锋铁骑'], ['黑暗潜能'], ['画地为牢'], ['守护机关'], ['狂兽血性'], ['龙鸣'], ['惊雷之龙'], ['破云之龙'], ['天翔之龙']], '获得': [['损失'], ['获得']], '效果': [['数值', '状态']], '数值': [['1'], ['1000'], ['5000'], ['100']], '状态': [['法力'], ['生命']]}
[['施法', '，', '结果', '。']]
[['主语', '动作', '技能']]
[['张飞'], ['关羽'], ['赵云'], ['典韦'], ['许褚'], ['刘备'], ['黄忠'], ['曹操'], ['鲁班七号'], ['貂蝉']]
[['施放'], ['使用'], ['召唤']]
[['一骑当千'], ['单刀赴会'], ['青龙偃月'], ['刀锋铁骑'], ['黑暗潜能'], ['画地为牢'], ['守护机关'], ['狂兽血性'], ['龙鸣'], ['惊雷之龙'], ['破云之龙'], ['天翔之龙']]
[['主语', '获得', '效果']]
[['张飞'], ['关羽'], ['赵云'], ['典韦'], ['许褚'], ['刘备'], ['黄忠'], ['曹操'], ['鲁班七号'], ['貂蝉']]
[['损失'], ['获得']]
[['损失'], ['获得']]
[['损失'], ['获得']]
[['数值', '状态']]
[['1'], ['1000'], ['5000'], ['100']]
[['法力'], ['生命']]
刘备使用龙鸣，赵云损失1生命。
[['施法', '，', '结果', '。']]
[['主语', '动作